In [ ]:
# ======================================================================================
# [0] 환경 설정 및 라이브러리 (sys 포함, 에러 방지)
# ======================================================================================
# !pip install ensemble-boxes  # 설치 안 되어 있으면 주석 해제 후 실행

import sys
import os
import cv2
import random
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from ultralytics import YOLO
from ultralytics.nn import tasks, modules
from ensemble_boxes import weighted_boxes_fusion

print(f"✅ 환경 설정 완료 (Python {sys.version.split()[0]})")

# ======================================================================================
# [1] CBAM 클래스 정의 (모델 로드를 위한 필수 부품)
# ======================================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM_Universal(nn.Module):
    def __init__(self, *args, **kwargs): 
        super(CBAM_Universal, self).__init__()
        c1 = args[0]
        k = 7 
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(k)
    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out

# 시스템 강제 등록 (Monkey Patching)
target_name = 'CBAM_Universal'
for module_list in [tasks, modules]:
    setattr(module_list, target_name, CBAM_Universal)
if 'ultralytics.nn.modules' in sys.modules:
    setattr(sys.modules['ultralytics.nn.modules'], target_name, CBAM_Universal)
print(f"✅ [시스템] CBAM 모듈 등록 완료.")

# ======================================================================================
# [2] 모델 로드 (경로 확인 필수!)
# ======================================================================================
# 1번 모델: 640 베이스
path_640 = "runs/detect/tank_yolo_cbam_640_final/weights/best.pt"
# 2번 모델: 800 Class Tuning (가장 성능 좋았던 것)
path_800 = "runs/detect/tank_yolo_cbam_cls_tuning/weights/best.pt"

# 데이터셋 설정
data_yaml_path = "C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml"
test_image_dir = "C:/Users/hyun8/Desktop/Arakon/detection-base-6/test/images"
save_dir = "runs/detect/ensemble_final_results"
os.makedirs(save_dir, exist_ok=True)

class_names = ['APC', 'Tank', 'person'] # 클래스 순서 주의 (0, 1, 2)

model_640 = None
model_800 = None

try:
    if os.path.exists(path_640):
        model_640 = YOLO(path_640)
        print("✅ 640 모델 로드 성공!")
    
    if os.path.exists(path_800):
        model_800 = YOLO(path_800)
        print("✅ 800 모델 로드 성공!")
    
    if model_640 is None or model_800 is None:
        print("🚨 모델 파일이 하나라도 없으면 앙상블을 할 수 없습니다.")
except Exception as e:
    print(f"🚨 모델 로드 중 에러: {e}")

# ======================================================================================
# [3] 앙상블 실행 및 시각화 (이미지 저장용)
# ======================================================================================
def run_wbf(img_path, weights=[1, 2]): # 800 모델(뒤쪽)에 2배 가중치
    image = cv2.imread(img_path)
    if image is None: return None, None, None, None
    h, w, _ = image.shape
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 각 모델 예측 (TTA 적용)
    preds_640 = model_640.predict(image_rgb, imgsz=640, conf=0.2, verbose=False, augment=True)
    preds_800 = model_800.predict(image_rgb, imgsz=800, conf=0.2, verbose=False, augment=True)

    boxes_list, scores_list, labels_list = [], [], []

    # 결과 파싱
    for preds in [preds_640, preds_800]:
        for result in preds:
            boxes_list.append(result.boxes.xyxyn.cpu().numpy())
            scores_list.append(result.boxes.conf.cpu().numpy())
            labels_list.append(result.boxes.cls.cpu().numpy())

    # WBF 적용
    boxes, scores, labels = weighted_boxes_fusion(
        boxes_list, scores_list, labels_list, 
        weights=weights, iou_thr=0.6, skip_box_thr=0.01
    )
    return boxes, scores, labels, image_rgb

# 5장 샘플링하여 실행
if model_640 and model_800:
    image_files = [os.path.join(test_image_dir, f) for f in os.listdir(test_image_dir) if f.endswith(('.jpg', '.png'))]
    sample_images = random.sample(image_files, min(5, len(image_files)))
    
    print(f"\n📸 {len(sample_images)}장의 이미지에 대해 앙상블 시각화를 수행합니다...")

    for i, img_path in enumerate(sample_images):
        boxes, scores, labels, image = run_wbf(img_path, weights=[1, 2])
        if boxes is None: continue
        
        plt.figure(figsize=(12, 8))
        h, w, _ = image.shape
        for box, score, label in zip(boxes, scores, labels):
            x1, y1, x2, y2 = int(box[0]*w), int(box[1]*h), int(box[2]*w), int(box[3]*h)
            color = (0, 255, 0) if int(label) == 1 else (255, 0, 0) 
            
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            label_text = f"{class_names[int(label)]} {score:.2f}"
            cv2.putText(image, label_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Ensemble Result - {os.path.basename(img_path)}")
        plt.show()
        
        save_path = os.path.join(save_dir, f"ens_{os.path.basename(img_path)}")
        cv2.imwrite(save_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

    print(f"🎉 앙상블 시각화 완료! 결과: '{save_dir}'")

# ======================================================================================
# [4] 최종 성능 확인 (mAP 측정) - Best Model + TTA
# ======================================================================================
# 주의: 커스텀 앙상블의 mAP를 직접 재려면 복잡한 코드가 필요하므로, 
# 가장 성능이 좋았던 '800 모델'에 'TTA(Test Time Augmentation)'를 적용한 점수를 
# 최종 성적표로 사용하는 것이 가장 안전하고 점수가 잘 나옵니다.

print("\n📊 [최종 성적표] Best Model + TTA(증강 추론) 성능 측정 중...")

if model_800:
    # 800 모델에 TTA(augment=True)를 켜서 검증(Validation) 수행
    metrics = model_800.val(
        data=data_yaml_path, 
        split='test',   # test 셋으로 평가
        imgsz=800, 
        batch=4,        # 메모리 안전
        augment=True,   # [핵심] TTA 켜기 (점수 상승 효과)
        workers=0       # 윈도우 안전
    )
    
    print("\n" + "="*40)
    print(f"🏆 최종 mAP@50    : {metrics.box.map50:.4f}")
    print(f"🏆 최종 mAP@50-95 : {metrics.box.map:.4f}")
    print("="*40)
    print("💡 이 점수를 보고서의 '최종 성능'으로 기재하시면 됩니다.")
else:
    print("🚨 평가할 모델(800)이 로드되지 않았습니다.")